# 상관분석

* internal_metrics() 이상치를 한번 파악하려고 봤더니 상자그림이 이상하게 그려졌다. 25
* knob 이상치 제거할 필요 없다. 랜덤생성했기 때문에 14 (+ 범주형은 10개)

1. 데이터 전처리
2. 상관분석 함수
3. activedefrag 있는 경우
4. 전체 knob
5. activedefrag 없는 경우
6. pointbeir~


len = 17
cluster0
hz
lru_clock
used_memory_peak
used_memory_overhead
allocator_allocated
allocator_active
maxmemory
maxmemory_human
allocator_frag_bytes
rss_overhead_ratio
rss_overhead_bytes
rdb_last_save_time
instantaneous_ops_per_sec
instantaneous_input_kbps
instantaneous_output_kbps
used_cpu_sys
used_cpu_user

len = 7
cluster1
used_memory_rss
used_memory_rss_human
allocator_resident
allocator_rss_ratio
allocator_rss_bytes
mem_fragmentation_ratio
mem_fragmentation_bytes

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# internal metrics 분류 결과
cluster0 =[ 
'uptime_in_seconds',
'hz',
'lru_clock',
'used_memory_rss',
'used_memory_rss_human',
'used_memory_peak_perc',
'used_memory_dataset_perc',
'maxmemory',
'maxmemory_human',
'allocator_frag_ratio',
'allocator_rss_ratio',
'rss_overhead_ratio',
'rss_overhead_bytes',
'mem_fragmentation_ratio',
'mem_fragmentation_bytes',
'rdb_last_save_time',
'rdb_last_bgsave_time_sec',
'rdb_current_bgsave_time_sec',
'rdb_last_cow_size',
'aof_current_size',
'keyspace_misses',
'latest_fork_usec',
'used_cpu_sys',
"used_cpu_user",
'used_cpu_sys_children',
'used_cpu_user_children',
]

cluster1 = [
'used_memory',
'used_memory_human',
'used_memory_peak',
'used_memory_peak_human',
'used_memory_overhead',
'used_memory_dataset',
'allocator_allocated',
'allocator_active',
'allocator_resident',
'allocator_frag_bytes',
'allocator_rss_bytes',
'mem_not_counted_for_evict',
'mem_aof_buffer',
'rdb_changes_since_last_save',
'aof_base_size',
'aof_buffer_length',
'instantaneous_ops_per_sec',
'total_net_output_bytes',
'instantaneous_input_kbps',
'instantaneous_output_kbps',
'db0:keys',
]

knob_list = [
    "hash-max-ziplist-entries", 
    "hash-max-ziplist-value",
    "activerehashing",  ##yes
    "hz",
    "dynamic-hz",
    
    # appendonly 제외
    'appendfsync',
    'auto-aof-rewrite-percentage',
    'auto-aof-rewrite-min-size',
    'no-appendfsync-on-rewrite',
    'aof-rewrite-incremental-fsync',
    'aof-use-rdb-preamble',
    
    "activedefrag",
    "active-defrag-threshold-lower",
    "active-defrag-threshold-upper",
    "active-defrag-cycle-min",
    "active-defrag-cycle-max",
    
    "maxmemory",   ##"volatile-lru"
    "maxmemory-policy",
    "maxmemory-samples",
    
    "lazyfree-lazy-eviction",
    "lazyfree-lazy-expire",
    "lazyfree-lazy-server-del"
]

activedefrag_knobs = [
    "activedefrag",
    "active-defrag-threshold-lower",
    "active-defrag-threshold-upper",
    "active-defrag-cycle-min",
    "active-defrag-cycle-max",
]

In [3]:
# 데이터 불러오기 internal metrics와 knob들의 상관관계분석
# internal_csv = pd.read_csv('./data/internal_metrcis0.csv')

file_dir = '../data/'
file_name = 'internal_metrics_r10000_aof1.csv'

internal_csv = pd.read_csv(file_dir+file_name)

# 1250 개 추출
internal_csv = internal_csv.iloc[0:1250,:]
internal_csv.shape

(1000, 131)

In [4]:
# role == master 인 부분 제외
idx_role_null = internal_csv['role'] == 'master'
internal_csv = internal_csv[idx_role_null]
internal_csv.shape

(1000, 131)

In [5]:
# 데이터 불러오기 knob_list
# knobs = pd.read_csv('./data/result_config0.csv')

file_dir = '../data/'
file_name = 'result_config_r10000_aof.csv'

knobs = pd.read_csv(file_dir + file_name)
# 1250 개 추출
knobs = knobs.iloc[0:1000,:]
knobs.shape

(1000, 32)

In [6]:
# role == master 인 부분 제외
knobs = knobs[idx_role_null]
knobs.shape

(1000, 32)

In [7]:
# 컬럼 추출 함수
def extract_data(cluster_list, df):
    extracted_df = pd.DataFrame()
    for elem in cluster_list:
        extracted_df[elem] = df[elem]

    return extracted_df    

## 데이터 전처리

In [8]:
# 클러스터 합치기
cluster = cluster0+cluster1

In [9]:
# 클러스터 요소에 해당되는 metrics 25개 추출
total_internal_df = extract_data(cluster, internal_csv)

### 결측치 확인

In [10]:
# internal_csv 결측치 확인 
total_internal_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 47 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   uptime_in_seconds            1000 non-null   int64  
 1   hz                           1000 non-null   int64  
 2   lru_clock                    1000 non-null   int64  
 3   used_memory_rss              1000 non-null   int64  
 4   used_memory_rss_human        1000 non-null   float64
 5   used_memory_peak_perc        1000 non-null   float64
 6   used_memory_dataset_perc     1000 non-null   float64
 7   maxmemory                    1000 non-null   int64  
 8   maxmemory_human              1000 non-null   float64
 9   allocator_frag_ratio         1000 non-null   float64
 10  allocator_rss_ratio          1000 non-null   float64
 11  rss_overhead_ratio           1000 non-null   float64
 12  rss_overhead_bytes           1000 non-null   int64  
 13  mem_fragmentation_r

In [11]:
# # # throughput 결측치 제거
# total_internal_df=total_internal_df.dropna()

In [12]:
# maxmemory 처리 knob
knobs['maxmemory'] = knobs['maxmemory'].str.strip('mb')  # 단위 자르기
knobs['maxmemory'] = pd.to_numeric(knobs['maxmemory'])  # 숫자형으로 바꾸기

# auto-aof-rewrite-min-size knob 단위(mb) 처리
knobs['auto-aof-rewrite-min-size']  = knobs['auto-aof-rewrite-min-size'].str.strip('mb')
knobs['auto-aof-rewrite-min-size'] = pd.to_numeric(knobs['auto-aof-rewrite-min-size'])

### activedefrag 처리
* subTotal_knobs_Active : 약 1300개 이며 activedefrag가 있는 경우 
* subTotal_knobs_NoActive: 약 1300개 이며 activedefrag가 nan값으로 제거한 경우
* subTotal_knobs_T : 2500개 이며 activedefrag 상관없이 전체 데이터


* total_internal_Active : 약 1300개 이며 activedefrag가 있는 경우 
* total_internal_NoActive: 약 1300개 이며 activedefrag가 nan값으로 제거한 경우
* total_internal_T : 2500개 이며 activedefrag 상관없이 전체 데이터

In [13]:
# knob_list 만 추출
subTotal_knobs = extract_data(knob_list, knobs)

In [14]:
#activedefrag knobs 분리

# activedefrag O 존재
subTotal_knobs_Active = subTotal_knobs.dropna()
Active_index = list(subTotal_knobs_Active.index)  # 인덱스

# # activedefrag X 존재하지 않는 나머지 반
subTotal_knobs_NoActive = subTotal_knobs.loc[subTotal_knobs['activedefrag'].isna()].dropna(axis=1)
NoActive_index = list(subTotal_knobs_NoActive.index)  # 인덱스

# activedefrag O and X 전체 
subTotal_knobs_T = subTotal_knobs.dropna(axis=1)
total_index = list(subTotal_knobs_T.index)   # 인덱스

In [15]:
# knobs에서 타입별로 분리 (수치, 불리언, 불리언아닌 문자)

def varType(subTotal_knobs):
    object_var = []
    numerical_var = []  # 문자, 숫자 타입
    
    for idx in subTotal_knobs.dtypes.index:
        if subTotal_knobs.dtypes[idx] == 'O':  # 문자일 경우
            object_var.append(idx)
        else: 
            numerical_var.append(idx)  # 숫자일 경우

    boolean_var = []
    booleanNo_var = []  # 2개, 2개이상

    for col in object_var:
        col_count = len(subTotal_knobs[col].unique())

        if col_count == 2:  # 2개일 경우
            boolean_var.append(col)
        elif col_count > 2:  # 2개 이상일 경우
            booleanNo_var.append(col)
    
    return numerical_var, boolean_var, booleanNo_var

# 수치, 불리언, 불리언아닌것 (전체)
numerical_var, boolean_var, NobooleanNo_var = varType(subTotal_knobs_T)

# 수치, 불리언, 불리언아닌것 (activedefrag)
numericalActive_var, booleanActive_var, NobooleanNoActive_var = varType(subTotal_knobs_Active)

# 수치, 불리언, 불리언아닌것 (activedefrag 없는것)
numericalNoActive_var, booleanNoActive_var, NobooleanNoActive_var = varType(subTotal_knobs_NoActive)


In [16]:
# internal에서 분리 

# activedefrag O
total_internal_Active = total_internal_df.loc[Active_index]

# activedefrag X
total_internal_NoActive = total_internal_df.loc[NoActive_index]

# activedefrag O and X
total_internal_T = total_internal_df.loc[total_index]

---

## 상관분석 함수

In [17]:
def get_outlier(df=None, column=None, weight=1.5):
    
    # target 값과 상관관계가 높은 열을 우선적으로 진행
    quantile_25 = np.percentile(df[column].values, 25)
    quantile_75 = np.percentile(df[column].values, 75)

    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*weight

    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight

    outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
    return outlier_idx

In [18]:
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau


# knob하나 internal 여러개 상관계수 구하기

# pearson
## 모수 검정
def cal_pearsonr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = pearsonr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# kendall
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 샘플사이저 적거나, 데이터 동률이 많을 때 유용
def cal_kendalltau(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = kendalltau(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# 스피어만
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 데이터 내 편차와 에러에 만감하며, 일반적으로 켄달보다 높은 값을 가진다.
def cal_spearmanr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = spearmanr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df


# knob하나 internal 여러개 상관계수 구하기 이상치 제거 버전
# pearson
## 모수 검정
def cal_pearsonr_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = pearsonr(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])

# kendall
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 샘플사이저 적거나, 데이터 동률이 많을 때 유용
def cal_kendalltau_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = kendalltau(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])

    
# 스피어만
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 데이터 내 편차와 에러에 만감하며, 일반적으로 켄달보다 높은 값을 가진다.
def cal_spearmanr_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = spearmanr(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])



In [19]:
# knob 하나별로 3가지 상관분석 수행
def extract_pvalue(knobs, internal_df):
    print('pearson')
    df = cal_pearsonr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()
    
    print('kendalltau')
    df = cal_kendalltau(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()
    
    print('spearman')
    df = cal_spearmanr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()

def extract_pvalue_outlier(knobs, internal_df):
    print('pearson')
    cal_pearsonr_outlier(knobs, internal_df)
    print()
    
    print('kendalltau')
    cal_kendalltau_outlier(knobs, internal_df)
    print()
    
    print('spearman')
    cal_spearmanr_outlier(knobs, internal_df)
    print()
    

# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)
        
        
# knob 리스트 각각 별로 extract_pvalue 수행 이상치 제거 
def extract_pvalue_knob_outlier(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue_outlier(subTotal_knobs[knob], total_internal)

---

## activedefrag 상관계수
* subTotal_knobs_Active
* total_internal_Active

### numericalActive_var :
* active-defrag-threshold-lower 
* active-defrag-threshold-upper 
* active-defrag-cycle-min
* active-defrag-cycle-max

In [20]:
numericalActive_var

['hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz',
 'auto-aof-rewrite-percentage',
 'auto-aof-rewrite-min-size',
 'active-defrag-threshold-lower',
 'active-defrag-threshold-upper',
 'active-defrag-cycle-min',
 'active-defrag-cycle-max',
 'maxmemory',
 'maxmemory-samples']

In [21]:
numericalActive_var
numericalActive_var_new = numericalActive_var[5:9]
numericalActive_var_new

['active-defrag-threshold-lower',
 'active-defrag-threshold-upper',
 'active-defrag-cycle-min',
 'active-defrag-cycle-max']

In [22]:
extract_pvalue_knob(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)

###active-defrag-threshold-lower###
pearson
                         corr   p-value
allocator_rss_ratio  0.097725  0.031948

kendalltau
                      corr   p-value
aof_current_size  0.081509  0.031146
aof_base_size     0.085017  0.024653

spearman
                      corr   p-value
aof_current_size  0.098278  0.030984
aof_base_size     0.102535  0.024373

###active-defrag-threshold-upper###
pearson


/Users/joseong-un/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/Users/joseong-un/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/Users/joseong-un/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###active-defrag-cycle-min###
pearson


/Users/joseong-un/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/Users/joseong-un/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


                              corr   p-value
used_memory_dataset_perc  0.092694  0.041937
used_memory              -0.101189  0.026318
used_memory_human        -0.101313  0.026133
used_memory_overhead     -0.097520  0.032310

kendalltau
                           corr   p-value
used_memory_peak_perc -0.090703  0.014081
used_memory_human     -0.085814  0.023328
used_memory_peak       0.087478  0.009806

spearman
                           corr   p-value
used_memory_peak_perc -0.111401  0.014404
used_memory_human     -0.103448  0.023125
used_memory_peak       0.116824  0.010260

###active-defrag-cycle-max###
pearson


/Users/joseong-un/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/Users/joseong-un/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


                                 corr   p-value
uptime_in_seconds           -0.090659  0.046667
rdb_last_bgsave_time_sec    -0.095375  0.036327
used_cpu_sys                -0.091938  0.043646
rdb_changes_since_last_save  0.104722  0.021478

kendalltau
                                 corr   p-value
used_memory_rss              0.064749  0.040120
used_memory_rss_human        0.068000  0.034198
mem_fragmentation_ratio      0.064302  0.043504
mem_fragmentation_bytes      0.066556  0.034029
rdb_last_bgsave_time_sec    -0.078492  0.038990
used_cpu_sys                -0.070318  0.024148
used_cpu_user               -0.066004  0.034307
used_cpu_user_children      -0.077764  0.028136
used_memory_peak            -0.080568  0.018185
allocator_resident           0.065398  0.041938
allocator_rss_bytes          0.063788  0.046688
rdb_changes_since_last_save  0.078909  0.025280

spearman
                                 corr   p-value
used_memory_rss              0.093673  0.039810
used_memory_rss_hu

/Users/joseong-un/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


In [23]:
# cluster 별 knob들의 상관계수 합  
active_result = pd.DataFrame(index = numericalActive_var_new)

cluster_list = [cluster0, cluster1]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in numericalActive_var_new:
        df = cal_pearsonr(subTotal_knobs_Active[knob], total_internal_Active.loc[:,cluster])
        cluster_val.append(df['corr'].abs().mean())

    active_result['cluster'+str(idx)] = cluster_val

/Users/joseong-un/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [24]:
active_result

,cluster0,cluster1
active-defrag-threshold-lower,0.030065,0.036516
active-defrag-threshold-upper,0.033583,0.026840
active-defrag-cycle-min,0.040898,0.042833
active-defrag-cycle-max,0.040553,0.038432


In [25]:
# knob이 어떤 클러스터에 속하는지 확인 가능 함수
def get_cluster(df):
    cluster_list = df.columns.tolist()  # [cluster0, cluster1]
    for knob_index in list(df.index):
        max_index = np.argmax(df.loc[knob_index,:].values)
        print(knob_index+' : ', end='')
        print(cluster_list[max_index])    

In [27]:
get_cluster(active_result)

active-defrag-threshold-lower : cluster1
active-defrag-threshold-upper : cluster0
active-defrag-cycle-min : cluster1
active-defrag-cycle-max : cluster0


### 이상치 제거 후 상관계수

In [28]:
# extract_pvalue_knob_outlier(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)

---

## 전체 knob
* subTotal_knobs_T
* total_internal_T

### numerical_var

In [29]:
numerical_var

['hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz',
 'auto-aof-rewrite-percentage',
 'auto-aof-rewrite-min-size',
 'maxmemory',
 'maxmemory-samples']

In [30]:
extract_pvalue_knob(numerical_var, subTotal_knobs_T,total_internal_T)

###hash-max-ziplist-entries###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###hash-max-ziplist-value###
pearson
               corr   p-value
maxmemory  0.069812  0.027272

kendalltau
                      corr   p-value
maxmemory         0.046627  0.031561
maxmemory_human   0.061224  0.004758
allocator_active  0.048784  0.049285

spearman
                      corr   p-value
maxmemory         0.068345  0.030689
maxmemory_human   0.089644  0.004554
allocator_active  0.062291  0.048923

###hz###
pearson
                                 corr       p-value
uptime_in_seconds            0.075935  1.631751e-02
hz                           1.000000  0.000000e+00
allocator_frag_ratio        -0.067155  3.372339e-02
rdb_current_bgsave_time_sec -0.128557  4.558712e-05
used_cpu_sys                 0.080949  1.044248e-02
used_cpu_user                0.137638  1

In [31]:
subTotal_knobs_T  # knobs에서 aof 관련 제거한 것
total_internal_T  # 전체

,uptime_in_seconds,hz,lru_clock,used_memory_rss,used_memory_rss_human,used_memory_peak_perc,used_memory_dataset_perc,maxmemory,maxmemory_human,allocator_frag_ratio,...,mem_not_counted_for_evict,mem_aof_buffer,rdb_changes_since_last_save,aof_base_size,aof_buffer_length,instantaneous_ops_per_sec,total_net_output_bytes,instantaneous_input_kbps,instantaneous_output_kbps,db0:keys
0,6,6,4246225,4259840,4.06,95.15,59.09,1048576000,1000.00,1.21,...,156,160,12744,205846,0,7530,100000,408.13,36.77,5000
1,6,18,4246245,4050944,3.86,95.15,59.08,1887436800,1.76,1.21,...,220,224,10000,0,0,16314,100000,884.23,79.67,5000
2,483,16,4246734,4104192,3.91,95.15,59.08,2621440000,2.44,1.21,...,220,224,3544,0,0,14,100000,0.79,0.08,5000
3,6,2,4246746,4075520,3.89,95.15,59.08,2202009600,2.05,1.21,...,220,224,10000,0,0,2495,100000,135.23,12.18,5000
4,483,7,4247235,3993600,3.81,95.15,59.08,1468006400,1.37,1.21,...,220,224,3785,0,0,31,100000,1.71,0.16,5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,6,5,4461639,4071424,3.88,95.15,59.08,1677721600,1.56,1.21,...,220,224,10000,0,0,6250,100000,338.75,30.52,5000
996,6,27,4461651,4079616,3.89,95.15,55.93,2516582400,2.34,1.21,...,220,224,10000,0,0,16854,100000,913.49,82.30,5000
997,6,24,4461664,4050944,3.86,95.15,59.09,1887436800,1.76,1.21,...,156,160,10000,0,0,15209,100000,824.34,74.26,5000
998,544,37,4462214,4182016,3.99,95.15,59.08,1048576000,1000.00,1.21,...,220,224,4376,0,0,18,100000,1.02,0.09,5000


In [32]:
# cluster 별 knob들의 상관계수 합  
numerical_result = pd.DataFrame(index = numerical_var)

cluster_list = [cluster0, cluster1]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in numerical_var:
        df = cal_spearmanr(subTotal_knobs_T[knob], total_internal_T.loc[:,cluster])
        cluster_val.append(df['corr'].abs().mean())

    numerical_result['cluster'+str(idx)] = cluster_val

In [33]:
numerical_result

,cluster0,cluster1
hash-max-ziplist-entries,0.020878,0.021634
hash-max-ziplist-value,0.023193,0.026211
hz,0.104122,0.092831
auto-aof-rewrite-percentage,0.027117,0.034750
auto-aof-rewrite-min-size,0.026535,0.026542
maxmemory,0.079543,0.023752
maxmemory-samples,0.031796,0.026476


In [35]:
get_cluster(numerical_result)

hash-max-ziplist-entries : cluster1
hash-max-ziplist-value : cluster1
hz : cluster0
auto-aof-rewrite-percentage : cluster1
auto-aof-rewrite-min-size : cluster1
maxmemory : cluster0
maxmemory-samples : cluster0


In [36]:
# numerical_result

In [37]:
# extract_pvalue_knob_outlier(numerical_var, subTotal_knobs_T,total_internal_T)

---

## Activedefrag 없는 knob들
* subTotal_knobs_Active
* total_internal_Active
active인거는 신경 안써도 된다.
### numericalActive_var

In [38]:
# numericalActive_var

In [39]:
# extract_pvalue_knob(numericalActive_var, subTotal_knobs_Active,total_internal_Active)

## other knob
* subTotal_knobs_NoActive
* total_internal_NoActive
active인거는 신경 안써도 된다.
### numericalNoActive_var

In [40]:
# numericalNoActive_var

In [41]:
# extract_pvalue_knob(numericalNoActive_var, subTotal_knobs_NoActive,total_internal_NoActive)

---

## pointbiserialr 분석

In [42]:
from scipy.stats import pointbiserialr
# pointbiserialr
## yes,no 데이터와 연속형 데이터 상관분석
def cal_pointbiserialr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = pointbiserialr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# knob 하나별로 3가지 상관분석 수행
def extract_pvalue(knobs, internal_df):
    print('pointbierialr')
    df = cal_pointbiserialr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()

    
# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob_point(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)


# 이상치 제거 버전
def cal_pointbiserialr_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = pointbiserialr(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])

# knob 하나별로 3가지 상관분석 수행
def extract_pvalue_outlier(knobs, internal_df):
    print('pointbierialr')
    cal_pointbiserialr(knobs, internal_df)
    print()

# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob_point_outlier(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)


## other knob(activedefrag 제외)
* subTotal_knobs_T
* total_internal_T

### boolean_var

In [43]:
boolean_var

['activerehashing',
 'dynamic-hz',
 'no-appendfsync-on-rewrite',
 'aof-rewrite-incremental-fsync',
 'aof-use-rdb-preamble',
 'lazyfree-lazy-eviction',
 'lazyfree-lazy-expire',
 'lazyfree-lazy-server-del']

In [44]:
#  yes, no -> 1, 0으로 변경
subTotal_knobs_T_bool = subTotal_knobs_T[boolean_var]  ## boolean 변수 df 추출
subTotal_knobs_T_point = subTotal_knobs_T_bool.replace(['yes', 'no'], [1,0])  ## 1, 0 변경

In [45]:
extract_pvalue_knob_point(boolean_var, subTotal_knobs_T_point, total_internal_T)

###activerehashing###
pointbierialr
                                 corr  p-value
rdb_current_bgsave_time_sec -0.081922  0.00955

###dynamic-hz###
pointbierialr
                               corr   p-value
maxmemory                 -0.083378  0.008341
allocator_rss_ratio        0.064300  0.042062
allocator_rss_bytes        0.066754  0.034802
instantaneous_ops_per_sec  0.089498  0.004621
instantaneous_input_kbps   0.089497  0.004622
instantaneous_output_kbps  0.089499  0.004621

###no-appendfsync-on-rewrite###
pointbierialr
                                 corr       p-value
uptime_in_seconds           -0.161148  3.005725e-07
rdb_last_bgsave_time_sec    -0.107870  6.334889e-04
rdb_current_bgsave_time_sec  0.078551  1.296443e-02
used_cpu_sys                -0.158845  4.442810e-07
used_cpu_user               -0.148853  2.271572e-06
rdb_changes_since_last_save  0.090925  4.006248e-03
instantaneous_ops_per_sec    0.100164  1.516600e-03
instantaneous_input_kbps     0.100166  1.516279e-03
i

In [46]:
# cluster 별 knob들의 상관계수 합
bool_result = pd.DataFrame(index = boolean_var)

cluster_list = [cluster0, cluster1]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in boolean_var:
        df = cal_pointbiserialr(subTotal_knobs_T_point[knob], total_internal_T.loc[:,cluster])
        cluster_val.append(df['corr'].abs().mean())

    bool_result['cluster'+str(idx)] = cluster_val

In [47]:
bool_result

,cluster0,cluster1
activerehashing,0.020390,0.026123
dynamic-hz,0.029571,0.037217
no-appendfsync-on-rewrite,0.042940,0.037318
aof-rewrite-incremental-fsync,0.026942,0.028639
aof-use-rdb-preamble,0.023416,0.028426
lazyfree-lazy-eviction,0.022655,0.032319
lazyfree-lazy-expire,0.026801,0.020382
lazyfree-lazy-server-del,0.021664,0.022184


In [48]:
get_cluster(bool_result)

activerehashing : cluster1
dynamic-hz : cluster1
no-appendfsync-on-rewrite : cluster0
aof-rewrite-incremental-fsync : cluster1
aof-use-rdb-preamble : cluster1
lazyfree-lazy-eviction : cluster1
lazyfree-lazy-expire : cluster0
lazyfree-lazy-server-del : cluster1


In [49]:
# #이상치 제거
# extract_pvalue_knob_point_outlier(boolean_var, subTotal_knobs_T_point, total_internal_T)

---

In [53]:
# extract_pvalue_knob(numerical_var, subTotal_knobs_T,total_internal_T)
# extract_pvalue_knob(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)
# extract_pvalue_knob_point(boolean_var, subTotal_knobs_T_point, total_internal_T)